# Visualização dos Resultados do Benchmark de LLMs

Este notebook tem como objetivo analisar e visualizar os resultados gerados pelo script `run_benchmark.py`. Ele carrega o arquivo `benchmark_results.csv` e cria uma série de gráficos para comparar o desempenho dos diferentes modelos de linguagem em termos de **pontuação (acurácia)** e **velocidade (duração)**.

### Seção 1: Configuração e Carregamento dos Dados

Primeiro, vamos importar as bibliotecas necessárias, definir o estilo dos gráficos e carregar os dados do arquivo CSV para um DataFrame do Pandas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Define um estilo visual mais agradável para os gráficos
sns.set_theme(style="whitegrid")

# --- CONFIGURAÇÃO ---
# Altere o nome do arquivo se o seu for diferente
caminho_arquivo = 'benchmark_results.csv'

# Verifica se o arquivo existe antes de carregar
if not os.path.exists(caminho_arquivo):
    print(f"Erro: O arquivo '{caminho_arquivo}' não foi encontrado.")
    print("Por favor, execute o benchmark primeiro ou verifique o caminho do arquivo.")
else:
    # Carrega os resultados do benchmark
    df = pd.read_csv(caminho_arquivo)
    
    # Exibe as primeiras linhas e informações sobre o DataFrame
    print("Dados carregados com sucesso!")
    print("\n--- Primeiras 5 linhas ---")
    display(df.head())
    
    print("\n--- Informações do DataFrame ---")
    df.info()

#### Engenharia de Atributos

A coluna `test_case_id` contém o tipo de tarefa (`get` ou `insert`). Vamos extrair essa informação para uma nova coluna chamada `tipo_tarefa` para facilitar análises futuras.

In [ ]:
if 'df' in locals():
    # Extrai 'get' ou 'insert' do ID do caso de teste
    df['tipo_tarefa'] = df['test_case_id'].apply(lambda x: 'get' if 'get' in x else 'insert')
    
    print("Coluna 'tipo_tarefa' criada com sucesso.")
    display(df.head())

### Seção 2: Visão Geral do Desempenho

Nesta seção, criamos gráficos que resumem o desempenho geral de cada modelo, fornecendo uma visão macro da pontuação e velocidade.

In [ ]:
# Gráfico 1: Pontuação Média Geral por Modelo
if 'df' in locals():
    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=df, x='model_name', y='score', ci=None, estimator=pd.np.mean, palette='viridis')
    
    plt.title('Pontuação Média Geral por Modelo', fontsize=16, fontweight='bold')
    plt.xlabel('Modelo', fontsize=12)
    plt.ylabel('Pontuação Média', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# Gráfico 2: Duração Média (Velocidade) por Modelo
if 'df' in locals():
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df, x='model_name', y='duration_seconds', ci=None, estimator=pd.np.mean, palette='plasma')
    
    plt.title('Duração Média (Velocidade) por Modelo', fontsize=16, fontweight='bold')
    plt.xlabel('Modelo', fontsize=12)
    plt.ylabel('Duração Média (segundos)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

### Seção 3: Análise de Eficiência (Pontuação vs. Velocidade)

Um bom modelo não é apenas preciso, mas também rápido. O gráfico de dispersão abaixo nos ajuda a visualizar o trade-off entre pontuação e velocidade. O modelo ideal estaria no quadrante superior esquerdo (alta pontuação, baixa duração).

In [ ]:
# Gráfico 3: Gráfico de Dispersão - Pontuação vs. Duração
if 'df' in locals():
    # Calcula as médias para cada modelo
    summary_df = df.groupby('model_name').agg(
        avg_score=('score', 'mean'),
        avg_duration=('duration_seconds', 'mean')
    ).reset_index()

    plt.figure(figsize=(12, 8))
    sns.scatterplot(data=summary_df, x='avg_duration', y='avg_score', hue='model_name', s=200, palette='deep')

    # Adiciona rótulos para cada ponto
    for i in range(summary_df.shape[0]):
        plt.text(x=summary_df.avg_duration[i]+0.01, y=summary_df.avg_score[i],
                 s=summary_df.model_name[i], fontdict=dict(color='black', size=10))

    plt.title('Gráfico de Eficiência: Pontuação Média vs. Duração Média', fontsize=16, fontweight='bold')
    plt.xlabel('Duração Média (segundos) - Mais rápido →', fontsize=12)
    plt.ylabel('Pontuação Média - Melhor →', fontsize=12)
    plt.legend(title='Modelos', bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Adiciona uma anotação sobre o quadrante ideal
    plt.annotate('Quadrante Ideal\n(Alta Pontuação, Baixa Duração)', 
                 xy=(summary_df.avg_duration.min(), summary_df.avg_score.max()), 
                 xytext=(summary_df.avg_duration.min()*1.5, summary_df.avg_score.max()*0.9), 
                 arrowprops=dict(facecolor='black', shrink=0.05),
                 fontsize=12, backgroundcolor='lightyellow')

    plt.tight_layout()
    plt.show()

### Seção 4: Análise Detalhada do Desempenho

Agora, vamos aprofundar a análise, segmentando o desempenho dos modelos por nível de dificuldade, consistência e tipo de tarefa.

In [ ]:
# Gráfico 4: Desempenho por Nível de Dificuldade
if 'df' in locals():
    plt.figure(figsize=(14, 7))
    sns.barplot(data=df, x='model_name', y='score', hue='difficulty',
                hue_order=['easy', 'medium', 'hard'], palette='coolwarm')
    
    plt.title('Desempenho por Nível de Dificuldade', fontsize=16, fontweight='bold')
    plt.xlabel('Modelo', fontsize=12)
    plt.ylabel('Pontuação Média', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Dificuldade')
    plt.tight_layout()
    plt.show()

O gráfico de caixas (boxplot) abaixo mostra a distribuição das pontuações para cada modelo. Ele é excelente para avaliar a **consistência**. Um modelo com uma caixa curta e poucos pontos fora (outliers) é mais consistente e previsível do que um com uma caixa longa.

In [ ]:
# Gráfico 5: Consistência do Desempenho (Box Plot)
if 'df' in locals():
    plt.figure(figsize=(14, 7))
    sns.boxplot(data=df, x='model_name', y='score', palette='pastel')
    
    plt.title('Consistência do Desempenho (Distribuição de Pontuações)', fontsize=16, fontweight='bold')
    plt.xlabel('Modelo', fontsize=12)
    plt.ylabel('Pontuação', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

In [ ]:
# Gráfico 6: Desempenho por Tipo de Tarefa (Get vs. Insert)
if 'df' in locals():
    plt.figure(figsize=(14, 7))
    sns.barplot(data=df, x='model_name', y='score', hue='tipo_tarefa', palette='Set2')
    
    plt.title('Desempenho por Tipo de Tarefa (Get vs. Insert)', fontsize=16, fontweight='bold')
    plt.xlabel('Modelo', fontsize=12)
    plt.ylabel('Pontuação Média', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Tipo de Tarefa')
    plt.tight_layout()
    plt.show()